In [ ]:
# --- 1. 라이브러리 임포트 ---
!pip install ta deap reservoirpy statsmodels yfinance tensorflow
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import TimeSeriesSplit
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from reservoirpy import nodes
import multiprocessing
import json
import os
from scipy.stats import ttest_ind
import random
from deap import base, creator, tools, algorithms

# ta 라이브러리에서 필요한 지표들
from ta.momentum import RSIIndicator, StochasticOscillator, WilliamsRIndicator
from ta.trend import MACD, ADXIndicator, CCIIndicator
from ta.volatility import BollingerBands

# --- DEAP Creator 클래스 정의 (전역 범위에서 1회만 실행) ---
if hasattr(creator, "FitnessMax"):
    del creator.FitnessMax
if hasattr(creator, "Individual"):
    del creator.Individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# --- 맞춤형 돌연변이 함수 ---
def custom_ga_mutate(individual, low, up, indpb):
    size = len(individual)
    for i in range(size):
        if random.random() < indpb:
            if i < 5:
                individual[i] = random.randint(low[i], up[i])
            else:
                individual[i] = random.uniform(low[i], up[i])
    return individual,

# --- 2. S&P 500 예측기 클래스 정의 ---
class SP500TrendPredictor:
    """
    S&P 500 지수의 주가 방향성을 예측하는 클래스.
    (ver 7.3: ESN 이름 중복 오류 수정)
    """
    def __init__(self, symbol='^GSPC', period='15y'):
        self.symbol = symbol
        self.period = period
        self.data = None
        self.features = None
        self.target = None
        self.common_idx = None
        self.feature_scaler = MinMaxScaler()
        self.optimal_params = {}

    def collect_data(self):
        print(f"데이터 수집 중: {self.symbol}")
        ticker = yf.Ticker(self.symbol)
        self.data = ticker.history(period=self.period).dropna()
        self._validate_data_quality()
        print(f"최종 데이터: {len(self.data)}일치")
        return self.data

    def _validate_data_quality(self):
        self.data['Returns'] = self.data['Close'].pct_change()
        m, s = self.data['Returns'].mean(), self.data['Returns'].std()
        th = 3 * s
        outliers = self.data[abs(self.data['Returns'] - m) > th].copy()
        if len(outliers):
            self.data['Volume_MA20'] = self.data['Volume'].rolling(20).mean()
            indices_to_remove = []
            for idx, row in outliers.iterrows():
                va = self.data['Volume_MA20'].loc[idx]
                ratio = row['Volume'] / va if va > 0 else 0
                if ratio <= 1.5:
                    indices_to_remove.append(idx)
            if indices_to_remove:
                self.data = self.data.drop(indices_to_remove)
                self.data['Returns'] = self.data['Close'].pct_change()
            self.data = self.data.drop('Volume_MA20', axis=1, errors='ignore')

    def optimize_technical_indicators_ga(self, population_size=20, generations=10):
        print("\n=== GA 기반 지표 최적화 ===")
        if os.path.exists("optimal_params.json"):
            with open("optimal_params.json", 'r') as f:
                self.optimal_params = json.load(f)
                print("저장된 최적 파라미터를 불러왔습니다:", self.optimal_params)
                return self.optimal_params

        tb = base.Toolbox(); pool = multiprocessing.Pool(); tb.register("map", pool.map)
        tb.register("attr_rsi", random.randint, 10, 20); tb.register("attr_macd_f", random.randint, 8, 15)
        tb.register("attr_macd_s", random.randint, 20, 30); tb.register("attr_macd_sig", random.randint, 7, 12)
        tb.register("attr_bb_p", random.randint, 15, 25); tb.register("attr_bb_std", random.uniform, 1.5, 2.5)
        tb.register("individual", tools.initCycle, creator.Individual, (tb.attr_rsi, tb.attr_macd_f, tb.attr_macd_s, tb.attr_macd_sig, tb.attr_bb_p, tb.attr_bb_std), n=1)
        tb.register("population", tools.initRepeat, list, tb.individual); tb.register("evaluate", self._evaluate_technical_params)
        tb.register("mate", tools.cxTwoPoint)

        low_bounds =  [10, 8, 20, 7, 15, 1.5]
        up_bounds =   [20, 15, 30, 12, 25, 2.5]
        tb.register("mutate", custom_ga_mutate, low=low_bounds, up=up_bounds, indpb=0.2)

        tb.register("select", tools.selTournament, tournsize=3)
        pop = tb.population(n=population_size); hof = tools.HallOfFame(1)
        algorithms.eaSimple(pop, tb, cxpb=0.5, mutpb=0.2, ngen=generations, halloffame=hof, verbose=True)
        pool.close()
        bi = hof[0]
        self.optimal_params = {'rsi': int(bi[0]), 'macd_f': int(bi[1]), 'macd_s': int(bi[2]), 'macd_sig': int(bi[3]), 'bb_p': int(bi[4]), 'bb_std': float(bi[5])}
        print("최적 파라미터:", self.optimal_params)
        with open("optimal_params.json", 'w') as f: json.dump(self.optimal_params, f)
        return self.optimal_params

    def _evaluate_technical_params(self, ind):
        p = {'rsi': ind[0], 'macd_f': ind[1], 'macd_s': ind[2], 'macd_sig': ind[3], 'bb_p': ind[4], 'bb_std': ind[5]}
        close = self.data['Close']; rsi_series = RSIIndicator(close, window=p['rsi']).rsi()
        macd_obj = MACD(close, window_slow=p['macd_s'], window_fast=p['macd_f'], window_sign=p['macd_sig'])
        macd_diff = macd_obj.macd_diff()
        bb_obj = BollingerBands(close, window=p['bb_p'], window_dev=p['bb_std'])
        bb_lower, bb_upper = bb_obj.bollinger_lband(), bb_obj.bollinger_hband()
        df_eval = pd.DataFrame({'close': close, 'rsi': rsi_series, 'macd_diff': macd_diff, 'bb_lower': bb_lower, 'bb_upper': bb_upper}).dropna()
        sig = np.zeros(len(df_eval), dtype=int)
        sig[(df_eval['rsi'] < 30) & (df_eval['macd_diff'] > 0) & (df_eval['close'] < df_eval['bb_lower'])] = 1
        sig[(df_eval['rsi'] > 70) & (df_eval['macd_diff'] < 0) & (df_eval['close'] > df_eval['bb_upper'])] = -1
        ret, pos, entry_price = [], 0, 0.0; close_eval = df_eval['close']
        for i in range(len(sig)):
            if sig[i] == 1 and pos == 0: pos, entry_price = 1, close_eval.iloc[i]
            elif sig[i] == -1 and pos == 1: ret.append((close_eval.iloc[i] - entry_price) / entry_price); pos = 0
        if pos == 1: ret.append((close_eval.iloc[-1] - entry_price) / entry_price)
        returns = np.array(ret)
        sharpe = (returns.mean() / returns.std() * np.sqrt(252)) if len(returns) > 1 and returns.std() > 0 else 0.0
        return (sharpe,)

    def create_features(self, use_optimized_params=True):
        print(f"\n=== 특성 엔지니어링 ({'최적' if use_optimized_params else '경험적'} 파라미터) ===")
        df = self.data.copy()
        params = self.optimal_params if use_optimized_params and self.optimal_params else {'rsi':14, 'macd_f':12, 'macd_s':26, 'macd_sig':9, 'bb_p':20, 'bb_std':2.0}
        print(f"사용된 파라미터: {params}")
        df['Returns'] = df['Close'].pct_change(); df['HL_Ratio'] = (df['High'] - df['Low']) / df['Close']
        df['Volume_Ratio'] = df['Volume'] / df['Volume'].rolling(20).mean()
        df['RSI'] = RSIIndicator(df['Close'], window=params['rsi']).rsi()
        macd = MACD(df['Close'], window_slow=params['macd_s'], window_fast=params['macd_f'], window_sign=params['macd_sig'])
        df['MACD_Hist'] = macd.macd_diff()
        bb = BollingerBands(df['Close'], window=params['bb_p'], window_dev=params['bb_std'])
        df['BB_Lower'] = bb.bollinger_lband(); df['BB_Upper'] = bb.bollinger_hband()
        df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
        st = StochasticOscillator(df['High'], df['Low'], df['Close']); df['Stoch_K'] = st.stoch()
        df.dropna(inplace=True)
        self.features = df.drop(columns=['Open','High','Low','Close','Volume','Dividends','Stock Splits', 'Returns', 'BB_Lower', 'BB_Upper'])
        return self.features

    def create_trend_target(self, delta_p=0.02, delta_t=10):
        prices = self.data['Close'].values; targets = []
        for i in range(len(prices) - delta_t):
            curr, future = prices[i], prices[i+1:i+delta_t+1]
            inc = max((p - curr) / curr for p in future) if len(future) > 0 else 0
            dec = min((p - curr) / curr for p in future) if len(future) > 0 else 0
            if inc >= delta_p: targets.append(2)
            elif dec <= -delta_p: targets.append(0)
            else: targets.append(1)
        targets.extend([1] * delta_t)
        self.target = pd.Series(targets, index=self.data.index)
        return self.target

    def prepare_sequences(self, sequence_length=60):
        if self.features is None or self.target is None: raise ValueError("특성과 타겟을 먼저 생성해주세요.")
        self.common_idx = self.features.index.intersection(self.target.index)
        features_aligned, target_aligned = self.features.loc[self.common_idx], self.target.loc[self.common_idx]
        scaled_features = self.feature_scaler.fit_transform(features_aligned)
        X, y = [], []
        for i in range(sequence_length, len(scaled_features)):
            X.append(scaled_features[i-sequence_length:i]); y.append(target_aligned.iloc[i])
        return np.array(X), np.array(y)

    def _create_esn_model(self):
        # [수정] 고정된 name 파라미터를 제거하여 라이브러리가 자동으로 고유 이름을 부여하도록 함
        return nodes.ESN(units=200, sr=0.877119108124829, leakage=0.2503273022024346, rc_connectivity= 0.2, input_scaling= 1.4739968769639693,
                         input_dim=self.features.shape[1], output_dim=3)

    def execute_full_validation_run(self, X, y, n_splits=10):
        """
        하나의 'Run'을 구성하는 Walk-Forward 교차 검증 전체를 ESN으로 수행.
        """
        print(f"  - ESN기반 Walk-Forward(k={n_splits}) 검증 시작...")
        tscv = TimeSeriesSplit(n_splits=n_splits)
        fold_scores = []

        for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
            model = self._create_esn_model()
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            X_train_esn, X_val_esn = X_train[:, -1, :], X_val[:, -1, :]
            y_train_oh = to_categorical(y_train, num_classes=3)

            esn_fitted = model.fit(X_train_esn, y_train_oh, warmup=10)
            y_pred = esn_fitted.run(X_val_esn).argmax(axis=1)

            score = f1_score(y_val, y_pred, average='macro', zero_division=0)
            fold_scores.append(score)
            print(f"    Fold {fold+1}/{n_splits} F1-Score: {score:.4f}")

        mean_score = np.mean(fold_scores)
        print(f"  - Walk-Forward 검증 완료. 이번 RUN의 대표 점수(평균 F1): {mean_score:.4f}")
        return mean_score


# --- 3. 메인 실행 함수 ---
def main():
    """
    중첩 교차 검증: ESN 모델을 사용하여 최종 전략 비교
    """
    print("=== S&P 500 트렌드 예측 시스템 (ESN 기반 중첩 교차 검증) ===")

    # --- 1. 사전 준비 단계 (1회만 실행) ---
    print("\n--- [단계 1] 데이터 수집 및 전처리 (1회 실행) ---")
    ga_optimizer = SP500TrendPredictor()
    ga_optimizer.collect_data()
    ga_optimizer.optimize_technical_indicators_ga(population_size=30, generations=15)
    base_data = ga_optimizer.data
    optimal_params = ga_optimizer.optimal_params

    predictor_opt = SP500TrendPredictor()
    predictor_opt.data = base_data
    predictor_opt.optimal_params = optimal_params
    predictor_opt.create_features(use_optimized_params=True)
    predictor_opt.create_trend_target()
    X_opt, y_opt = predictor_opt.prepare_sequences()

    predictor_emp = SP500TrendPredictor()
    predictor_emp.data = base_data
    predictor_emp.create_features(use_optimized_params=False)
    predictor_emp.create_trend_target()
    X_emp, y_emp = predictor_emp.prepare_sequences()

    num_runs = 10

    # --- 2. 최적화 파라미터 전략 검증 (외부 루프) ---
    print(f"\n--- [단계 2] 최적화 파라미터 전략 검증 (ESN, {num_runs}회 반복) ---")
    all_opt_avg_scores = []
    for i in range(num_runs):
        print(f"\n>> 최적화 전략 RUN {i+1}/{num_runs} <<")
        avg_score = predictor_opt.execute_full_validation_run(X_opt, y_opt, n_splits=5)
        all_opt_avg_scores.append(avg_score)

    # --- 3. 경험적 파라미터 전략 검증 (외부 루프) ---
    print(f"\n--- [단계 3] 경험적 파라미터 전략 검증 (ESN, {num_runs}회 반복) ---")
    all_emp_avg_scores = []
    for i in range(num_runs):
        print(f"\n>> 경험적 전략 RUN {i+1}/{num_runs} <<")
        avg_score = predictor_emp.execute_full_validation_run(X_emp, y_emp, n_splits=5)
        all_emp_avg_scores.append(avg_score)

    # --- 4. 최종 결과 분석 ---
    print("\n\n" + "="*40 + "\n>>> 최종 중첩 검증 결과 요약 (ESN 기반) <<<\n" + "="*40)
    mean_opt, std_opt = np.mean(all_opt_avg_scores), np.std(all_opt_avg_scores)
    mean_emp, std_emp = np.mean(all_emp_avg_scores), np.std(all_emp_avg_scores)

    print(f"\n최적화 파라미터 전략 ({num_runs}회 실행 평균):")
    print(f"  - 대표 Macro F1-Score: {mean_opt:.4f} ± {std_opt:.4f}")

    print(f"\n경험적 파라미터 전략 ({num_runs}회 실행 평균):")
    print(f"  - 대표 Macro F1-Score: {mean_emp:.4f} ± {std_emp:.4f}")

    print("\n\n--- 최종 전략 통계적 유의성 검증 (Independent Samples t-test) ---")
    t_stat, p_value = ttest_ind(all_opt_avg_scores, all_emp_avg_scores, equal_var=False, alternative='greater')

    print(f"T-statistic: {t_stat:.4f}\nP-value: {p_value:.4f}")
    alpha = 0.05
    print(f"\n유의수준(α) = {alpha}")
    if p_value < alpha:
        print(f"결론: p-value({p_value:.4f}) < {alpha} 이므로 귀무가설을 기각합니다.")
        print(">> ESN 모델을 사용한 검증 하에, GA 파라미터 최적화는 통계적으로 유의미한 성능 향상을 가져왔습니다. ✅")
    else:
        print(f"결론: p-value({p_value:.4f}) >= {alpha} 이므로 귀무가설을 기각할 수 없습니다.")
        print(">> ESN 모델을 사용한 검증 하에, 파라미터 최적화의 성능 향상이 통계적으로 유의미하다고 말하기 어렵습니다. ❌")


# --- 4. 스크립트 실행 ---
if __name__ == "__main__":
    multiprocessing.freeze_support()
    main()

=== S&P 500 트렌드 예측 시스템 (ESN 기반 중첩 교차 검증) ===

--- [단계 1] 데이터 수집 및 전처리 (1회 실행) ---
데이터 수집 중: ^GSPC
최종 데이터: 3727일치

=== GA 기반 지표 최적화 ===
저장된 최적 파라미터를 불러왔습니다: {'rsi': 16, 'macd_f': 15, 'macd_s': 23, 'macd_sig': 12, 'bb_p': 17, 'bb_std': 2.416047958664179}

=== 특성 엔지니어링 (최적 파라미터) ===
사용된 파라미터: {'rsi': 16, 'macd_f': 15, 'macd_s': 23, 'macd_sig': 12, 'bb_p': 17, 'bb_std': 2.416047958664179}

=== 특성 엔지니어링 (경험적 파라미터) ===
사용된 파라미터: {'rsi': 14, 'macd_f': 12, 'macd_s': 26, 'macd_sig': 9, 'bb_p': 20, 'bb_std': 2.0}

--- [단계 2] 최적화 파라미터 전략 검증 (ESN, 10회 반복) ---

>> 최적화 전략 RUN 1/10 <<
  - ESN기반 Walk-Forward(k=5) 검증 시작...


Running ESN-4165: 100%|██████████| 1/1 [00:00<00:00, 10.72it/s]


Fitting node ESN-4165...


Running ESN-4165: 100%|██████████| 1/1 [00:00<00:00,  8.94it/s]


    Fold 1/5 F1-Score: 0.3726


...

    Fold 4/5 F1-Score: 0.4008


Running ESN-4264: 100%|██████████| 1/1 [00:00<00:00,  2.19it/s]


Fitting node ESN-4264...


Running ESN-4264: 100%|██████████| 1/1 [00:00<00:00,  8.97it/s]

    Fold 5/5 F1-Score: 0.3934
  - Walk-Forward 검증 완료. 이번 RUN의 대표 점수(평균 F1): 0.4172


>>> 최종 중첩 검증 결과 요약 (ESN 기반) <<<

최적화 파라미터 전략 (10회 실행 평균):
  - 대표 Macro F1-Score: 0.4147 ± 0.0098

경험적 파라미터 전략 (10회 실행 평균):
  - 대표 Macro F1-Score: 0.4207 ± 0.0058


--- 최종 전략 통계적 유의성 검증 (Independent Samples t-test) ---
T-statistic: -1.5718
P-value: 0.9313

유의수준(α) = 0.05
결론: p-value(0.9313) >= 0.05 이므로 귀무가설을 기각할 수 없습니다.
>> ESN 모델을 사용한 검증 하에, 파라미터 최적화의 성능 향상이 통계적으로 유의미하다고 말하기 어렵습니다. ❌


In [ ]:
# --- 1. 라이브러리 임포트 ---
!pip install ta deap reservoirpy statsmodels yfinance tensorflow
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score
# [수정] train_test_split 대신 TimeSeriesSplit 임포트
from sklearn.model_selection import TimeSeriesSplit
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional, BatchNormalization, Input
from tensorflow.keras.utils import to_categorical
from reservoirpy import nodes
import json
import os
from scipy.stats import ttest_ind

# ta 라이브러리에서 필요한 지표들
from ta.momentum import RSIIndicator, StochasticOscillator, WilliamsRIndicator
from ta.trend import MACD, ADXIndicator, CCIIndicator
from ta.volatility import BollingerBands

# --- 2. 데이터 및 특성 생성을 위한 헬퍼 클래스 (이전과 동일) ---
class DataProvider:
    def __init__(self, symbol='^GSPC', period='15y'):
        self.symbol = symbol
        self.period = period
        self.data = None
        self.features = None
        self.target = None
        self.feature_scaler = MinMaxScaler()

    def get_prepared_data(self, use_optimized_params=False, sequence_length=60):
        """데이터 수집부터 시퀀스 생성까지 모든 과정을 수행하고 X, y를 반환"""
        self._collect_data()
        self._create_features(use_optimized_params=use_optimized_params)
        self._create_trend_target()
        return self._prepare_sequences(sequence_length=sequence_length)

    def _collect_data(self):
        print(f"데이터 수집 중: {self.symbol}...")
        ticker = yf.Ticker(self.symbol)
        self.data = ticker.history(period=self.period).dropna()

    def _create_features(self, use_optimized_params):
        print("특성 엔지니어링 중...")
        df = self.data.copy()
        params = {'rsi':14, 'macd_f':12, 'macd_s':26, 'macd_sig':9, 'bb_p':20, 'bb_std':2.0}

        df['HL_Ratio'] = (df['High'] - df['Low']) / df['Close']
        df['Volume_Ratio'] = df['Volume'] / df['Volume'].rolling(20).mean()
        df['RSI'] = RSIIndicator(df['Close'], window=params['rsi']).rsi()
        macd = MACD(df['Close'], window_slow=params['macd_s'], window_fast=params['macd_f'], window_sign=params['macd_sig'])
        df['MACD_Hist'] = macd.macd_diff()

        bb = BollingerBands(df['Close'], window=params['bb_p'], window_dev=params['bb_std'])
        df['BB_Lower'] = bb.bollinger_lband()
        df['BB_Upper'] = bb.bollinger_hband()
        df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])

        st = StochasticOscillator(df['High'], df['Low'], df['Close'])
        df['Stoch_K'] = st.stoch()

        df.dropna(inplace=True)
        self.features = df[['HL_Ratio', 'Volume_Ratio', 'RSI', 'MACD_Hist', 'BB_Position', 'Stoch_K']]

    def _create_trend_target(self, delta_p=0.02, delta_t=10):
        prices = self.data['Close'].values; targets = []
        for i in range(len(prices) - delta_t):
            curr, future = prices[i], prices[i+1:i+delta_t+1]
            inc = max((p - curr) / curr for p in future) if len(future) > 0 else 0
            dec = min((p - curr) / curr for p in future) if len(future) > 0 else 0
            if inc >= delta_p: targets.append(2)
            elif dec <= -delta_p: targets.append(0)
            else: targets.append(1)
        targets.extend([1] * delta_t)
        self.target = pd.Series(targets, index=self.data.index)

    def _prepare_sequences(self, sequence_length):
        common_idx = self.features.index.intersection(self.target.index)
        features_aligned, target_aligned = self.features.loc[common_idx], self.target.loc[common_idx]
        scaled_features = self.feature_scaler.fit_transform(features_aligned)
        X, y = [], []
        for i in range(sequence_length, len(scaled_features)):
            X.append(scaled_features[i-sequence_length:i]); y.append(target_aligned.iloc[i])
        return np.array(X), np.array(y)

# --- 3. 모델 생성 팩토리 (이전과 동일) ---
def create_lstm_model(input_shape):
    model_input = Input(shape=input_shape)
    y = LSTM(40, return_sequences=True)(model_input); y = Dropout(0.2)(y)
    y = LSTM(40, return_sequences=False)(y); y = Dropout(0.2)(y)
    y = Dense(25)(y)
    model_output = Dense(3, activation='softmax')(y)
    model = Model(inputs=model_input, outputs=model_output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_gru_model(input_shape):
    gru_input = Input(shape=input_shape)
    y = Bidirectional(GRU(64, return_sequences=True))(gru_input); y = Dropout(0.3)(y)
    y = Bidirectional(GRU(64, return_sequences=False))(y); y = BatchNormalization()(y)
    y = Dense(64, activation='relu')(y); y = Dropout(0.4)(y)
    gru_output = Dense(3, activation='softmax')(y)
    model = Model(inputs=gru_input, outputs=gru_output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_esn_model(input_shape):
    return nodes.ESN(units=300, sr=0.8, leakage=0.7, input_dim=input_shape[1], output_dim=3)

# --- 4. 메인 실행 함수 ---
def main():
    """
    [수정] 시계열 교차 검증을 사용하여 최적의 모델 아키텍처를 탐색하는 워크플로우.
    """
    print("="*50)
    print("S&P 500 예측: 시계열 교차 검증을 이용한 모델 아키텍처 탐색")
    print("="*50)

    # --- 1. 데이터 준비 (1회 실행) ---
    provider = DataProvider()
    X, y = provider.get_prepared_data()
    print(f"\n전체 데이터 준비 완료. 데이터 포인트 수: {len(X)}")

    # --- 2. [핵심] 시계열 교차 검증 설정 ---
    n_splits = 10  # 10개의 폴드로 교차 검증 수행
    tscv = TimeSeriesSplit(n_splits=n_splits)

    models_to_evaluate = ['LSTM', 'GRU', 'ESN']
    # 각 모델의 폴드별 점수를 저장할 딕셔너리
    model_scores_cv = {name: [] for name in models_to_evaluate}

    input_shape = (X.shape[1], X.shape[2])

    # --- 3. [핵심] 모델별 시계열 교차 검증 실행 ---
    for model_name in models_to_evaluate:
        print(f"\n--- {model_name} 모델 시계열 교차 검증 (K={n_splits}) ---")

        # tscv.split(X)는 시간 순서에 따라 train/validation 인덱스를 생성
        for fold, (train_index, val_index) in enumerate(tscv.split(X)):
            tf.keras.backend.clear_session()

            # 현재 폴드에 맞는 훈련/검증 데이터 분할
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            if model_name == 'LSTM':
                model = create_lstm_model(input_shape)
                model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
                y_pred = np.argmax(model.predict(X_val, verbose=0), axis=1)

            elif model_name == 'GRU':
                model = create_gru_model(input_shape)
                model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
                y_pred = np.argmax(model.predict(X_val, verbose=0), axis=1)

            elif model_name == 'ESN':
                model = create_esn_model(input_shape)
                # ESN은 2D 입력을 기대하므로 시퀀스의 마지막 타임스텝만 사용
                X_train_esn, X_val_esn = X_train[:, -1, :], X_val[:, -1, :]
                y_train_oh = to_categorical(y_train, num_classes=3)
                esn_fitted = model.fit(X_train_esn, y_train_oh, warmup=10)
                y_pred = esn_fitted.run(X_val_esn).argmax(axis=1)

            # 현재 폴드의 성능(F1-score) 계산
            score = f1_score(y_val, y_pred, average='macro', zero_division=0)
            model_scores_cv[model_name].append(score)
            print(f"  Fold {fold+1}/{n_splits} | Train: {len(X_train)}, Val: {len(X_val)} | F1-Score: {score:.4f}")

    # --- 4. 최종 결과 분석 및 통계 검증 ---
    print("\n\n" + "="*50)
    print("최종 결과 요약 및 통계 검증 (시계열 교차 검증 기반)")
    print("="*50)

    for model_name, scores in model_scores_cv.items():
        print(f"\n{model_name} 모델 ({n_splits}-Fold CV 평균):")
        print(f"  - Macro F1-Score: {np.mean(scores):.4f} ± {np.std(scores):.4f}")

    print("\n--- 모델 간 성능 비교 (Independent Samples t-test) ---")
    model_names = list(model_scores_cv.keys())
    for i in range(len(model_names)):
        for j in range(i + 1, len(model_names)):
            model1_name, model2_name = model_names[i], model_names[j]
            scores1, scores2 = model_scores_cv[model1_name], model_scores_cv[model2_name]

            t_stat, p_value = ttest_ind(scores1, scores2, equal_var=False)

            mean1, mean2 = np.mean(scores1), np.mean(scores2)
            conclusion = "통계적으로 유의미한 차이 없음."
            if p_value < 0.05:
                winner = model1_name if mean1 > mean2 else model2_name
                conclusion = f"유의미한 차이 발견. ({winner} 우세)"
            print(f"  - {model1_name} vs {model2_name}: p-value = {p_value:.4f}. 결론: {conclusion}")

    best_model_name = max(model_scores_cv, key=lambda name: np.mean(model_scores_cv[name]))
    print(f"\n\n>> 최종 결론: 현 데이터셋과 교차 검증 결과, '{best_model_name}' 모델이 가장 안정적이고 우수한 성능을 보입니다. <<")


if __name__ == '__main__':
    main()

S&P 500 예측: 시계열 교차 검증을 이용한 모델 아키텍처 탐색
데이터 수집 중: ^GSPC...
특성 엔지니어링 중...

전체 데이터 준비 완료. 데이터 포인트 수: 3682

--- LSTM 모델 시계열 교차 검증 (K=10) ---
  Fold 1/10 | Train: 342, Val: 334 | F1-Score: 0.3454
  Fold 2/10 | Train: 676, Val: 334 | F1-Score: 0.4572
  Fold 3/10 | Train: 1010, Val: 334 | F1-Score: 0.3613
  Fold 4/10 | Train: 1344, Val: 334 | F1-Score: 0.4115
  Fold 5/10 | Train: 1678, Val: 334 | F1-Score: 0.4138
  Fold 6/10 | Train: 2012, Val: 334 | F1-Score: 0.4074
  Fold 7/10 | Train: 2346, Val: 334 | F1-Score: 0.4035
  Fold 8/10 | Train: 2680, Val: 334 | F1-Score: 0.3775
  Fold 9/10 | Train: 3014, Val: 334 | F1-Score: 0.3705
  Fold 10/10 | Train: 3348, Val: 334 | F1-Score: 0.4045

--- GRU 모델 시계열 교차 검증 (K=10) ---
  Fold 1/10 | Train: 342, Val: 334 | F1-Score: 0.2410
  Fold 2/10 | Train: 676, Val: 334 | F1-Score: 0.4167
  Fold 3/10 | Train: 1010, Val: 334 | F1-Score: 0.3941
  Fold 4/10 | Train: 1344, Val: 334 | F1-Score: 0.3381
  Fold 5/10 | Train: 1678, Val: 334 | F1-Score: 0.4389
  Fold 6/

Running ESN-90: 100%|██████████| 1/1 [00:00<00:00, 23.97it/s]


Fitting node ESN-90...


Running ESN-90: 100%|██████████| 1/1 [00:00<00:00, 12.51it/s]


  Fold 1/10 | Train: 342, Val: 334 | F1-Score: 0.3760


...

  Fold 9/10 | Train: 3014, Val: 334 | F1-Score: 0.4216


Running ESN-99: 100%|██████████| 1/1 [00:00<00:00,  2.67it/s]


Fitting node ESN-99...


Running ESN-99: 100%|██████████| 1/1 [00:00<00:00, 12.28it/s]

  Fold 10/10 | Train: 3348, Val: 334 | F1-Score: 0.4256


최종 결과 요약 및 통계 검증 (시계열 교차 검증 기반)

LSTM 모델 (10-Fold CV 평균):
  - Macro F1-Score: 0.3953 ± 0.0305

GRU 모델 (10-Fold CV 평균):
  - Macro F1-Score: 0.3727 ± 0.0680

ESN 모델 (10-Fold CV 평균):
  - Macro F1-Score: 0.4238 ± 0.0364

--- 모델 간 성능 비교 (Independent Samples t-test) ---
  - LSTM vs GRU: p-value = 0.3818. 결론: 통계적으로 유의미한 차이 없음.
  - LSTM vs ESN: p-value = 0.0891. 결론: 통계적으로 유의미한 차이 없음.
  - GRU vs ESN: p-value = 0.0675. 결론: 통계적으로 유의미한 차이 없음.


>> 최종 결론: 현 데이터셋과 교차 검증 결과, 'ESN' 모델이 가장 안정적이고 우수한 성능을 보입니다. <<


In [ ]:
# tune_esn_final.py
!pip install optuna
import pandas as pd
import numpy as np
import yfinance as yf
import optuna
import warnings
import os
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.utils import to_categorical
from reservoirpy import nodes
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands

warnings.filterwarnings("ignore", category=FutureWarning)


# ------------------- 데이터 준비 클래스 -------------------
class DataProvider:
    """데이터 수집 및 특성 생성을 위한 헬퍼 클래스"""
    def __init__(self, symbol='^GSPC', period='15y'):
        self.symbol = symbol
        self.period = period
        self.data = None
        self.feature_scaler = MinMaxScaler()

    def get_prepared_data(self, use_ga_features=False, sequence_length=60):
        self._collect_data()

        indicator_params = {'rsi':14, 'macd_f':12, 'macd_s':26, 'macd_sig':9, 'bb_p':20, 'bb_std':2.0}
        if use_ga_features:
            if os.path.exists("optimal_params.json"):
                 with open("optimal_params.json", 'r') as f: indicator_params = json.load(f)
                 print("저장된 GA 최적화 지표 파라미터를 사용합니다.")
            else:
                 print("GA 최적화 파라미터 파일(optimal_params.json)이 없어 경험적 파라미터를 사용합니다.")

        features = self._create_features(indicator_params)
        target = self._create_trend_target()
        return self._prepare_sequences(features, target, sequence_length)

    def _collect_data(self):
        print(f"데이터 수집 중: {self.symbol}...")
        self.data = yf.Ticker(self.symbol).history(period=self.period).dropna()

    def _create_features(self, params):
        print(f"특성 엔지니어링 중... (사용 파라미터: {params})")
        df = self.data.copy()
        df['Returns'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)
        df['HL_Ratio'] = (df['High'] - df['Low']) / df['Close']
        df['RSI'] = RSIIndicator(df['Close'], window=int(params['rsi'])).rsi()
        macd = MACD(df['Close'], window_slow=int(params['macd_s']), window_fast=int(params['macd_f']), window_sign=int(params['macd_sig']))
        df['MACD_Hist'] = macd.macd_diff()
        bb = BollingerBands(df['Close'], window=int(params['bb_p']), window_dev=params['bb_std'])
        df['BB_Position'] = (df['Close'] - bb.bollinger_lband()) / (bb.bollinger_hband() - bb.bollinger_lband())
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(inplace=True)
        return df[['Returns', 'HL_Ratio', 'RSI', 'MACD_Hist', 'BB_Position']]

    def _create_trend_target(self, delta_p=0.02, delta_t=10):
        prices = self.data['Close'].values
        targets = []
        for i in range(len(prices) - delta_t):
            curr, future = prices[i], prices[i+1 : i+delta_t+1]
            inc = max((p - curr) / curr for p in future) if len(future) > 0 else 0
            dec = min((p - curr) / curr for p in future) if len(future) > 0 else 0
            if inc >= delta_p: targets.append(2)
            elif dec <= -delta_p: targets.append(0)
            else: targets.append(1)
        targets.extend([1] * delta_t)
        return pd.Series(targets, index=self.data.index)

    def _prepare_sequences(self, features, target, sequence_length):
        common_idx = features.index.intersection(target.index)
        features_aligned, target_aligned = features.loc[common_idx], target.loc[common_idx]
        scaled_features = self.feature_scaler.fit_transform(features_aligned)
        X, y = [], []
        for i in range(sequence_length, len(scaled_features)):
            X.append(scaled_features[i-sequence_length:i])
            y.append(target_aligned.iloc[i])
        return np.array(X), np.array(y)

# ------------------- Optuna Objective 함수 -------------------
def objective(trial, X, y):
    """Optuna가 최적화할 목표 함수"""
    # 1. 하이퍼파라미터 탐색 공간 정의
    params = {
        'units': trial.suggest_int('units', 100, 800, step=50),
        'sr': trial.suggest_float('sr', 0.8, 1.5),
        'leakage': trial.suggest_float('leakage', 0.1, 1.0),
        'rc_connectivity': trial.suggest_float('rc_connectivity', 0.05, 0.5, step=0.05),
        'input_scaling': trial.suggest_float('input_scaling', 0.1, 2.0)
    }

    # 2. 시계열 교차 검증으로 모델 성능 평가
    n_splits = 5  # 교차 검증 폴드 수
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_scores = []

    for train_idx, val_idx in tscv.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        X_train_esn, X_val_esn = X_train[:, -1, :], X_val[:, -1, :]
        y_train_oh = to_categorical(y_train, num_classes=3)

        model = nodes.ESN(output_dim=3, seed=1234, **params)
        try:
            fitted_model = model.fit(X_train_esn, y_train_oh, warmup=10)
            y_pred = fitted_model.run(X_val_esn).argmax(axis=1)
            score = f1_score(y_val, y_pred, average='macro', zero_division=0)
            fold_scores.append(score)
        except np.linalg.LinAlgError:
            # 불안정한 파라미터 조합으로 인한 오류 발생 시, 낮은 점수 반환
            return 0.0

    # 3. 교차 검증 평균 점수를 최종 점수로 반환
    return np.mean(fold_scores)

# ------------------- 메인 실행부 -------------------
if __name__ == "__main__":

    # --- 1. 튜닝 환경 설정 ---
    USE_GA_FEATURES_FOR_TUNING = True

    # --- 2. 데이터 준비 ---
    provider = DataProvider()
    X_tune, y_tune = provider.get_prepared_data(use_ga_features=USE_GA_FEATURES_FOR_TUNING)
    print(f"\n튜닝용 데이터 준비 완료. X shape: {X_tune.shape}")

    # --- 3. Optuna 튜닝 실행 ---
    print("\n--- Optuna를 이용한 ESN 하이퍼파라미터 튜닝 시작 ---")

    # F1-Score를 '최대화(maximize)'하는 방향으로 study 객체 생성
    study = optuna.create_study(direction="maximize")


    study.optimize(
        lambda trial: objective(trial, X_tune, y_tune),
        n_trials=200,
        n_jobs=-1,
        show_progress_bar=True
    )

    # --- 4. 최종 결과 출력 ---
    print("\n\n" + "="*50)
    print("      ESN 하이퍼파라미터 튜닝 최종 결과")
    print("="*50)

    best_trial = study.best_trial
    print(f"\n 최고 성능 (Macro F1-Score): {best_trial.value:.4f}")

    print("\n 최적 하이퍼파라미터 조합:")
    for key, value in best_trial.params.items():
        print(f"  - {key:<15}: {value}")

    print("="*50)

데이터 수집 중: ^GSPC...
저장된 GA 최적화 지표 파라미터를 사용합니다.
특성 엔지니어링 중... (사용 파라미터: {'rsi': 16, 'macd_f': 15, 'macd_s': 23, 'macd_sig': 12, 'bb_p': 17, 'bb_std': 2.416047958664179})


[I 2025-06-14 13:06:40,304] A new study created in memory with name: no-name-f322a58e-0d95-4b4c-bab7-66c477841b61



튜닝용 데이터 준비 완료. X shape: (3682, 60, 5)

--- Optuna를 이용한 ESN 하이퍼파라미터 튜닝 시작 ---


  0%|          | 0/200 [00:00<?, ?it/s]


Running ESN-2965: 100%|██████████| 1/1 [00:00<00:00,  3.01it/s]


...

Fitting node ESN-3963...



Running ESN-3963: 100%|██████████| 1/1 [00:00<00:00,  6.58it/s]

Running ESN-3964: 100%|██████████| 1/1 [00:00<00:00,  1.74it/s]


Fitting node ESN-3964...



Running ESN-3964: 100%|██████████| 1/1 [00:00<00:00,  6.14it/s]


[I 2025-06-14 13:29:22,161] Trial 199 finished with value: 0.4310301259512731 and parameters: {'units': 200, 'sr': 0.9392796907721512, 'leakage': 0.13283606956742655, 'rc_connectivity': 0.2, 'input_scaling': 1.7236681035498542}. Best is trial 95 with value: 0.4350317041046913.


      ESN 하이퍼파라미터 튜닝 최종 결과

🚀 최고 성능 (Macro F1-Score): 0.4350

💡 최적의 하이퍼파라미터 조합:
  - units          : 200
  - sr             : 0.877119108124829
  - leakage        : 0.2503273022024346
  - rc_connectivity: 0.2
  - input_scaling  : 1.4739968769639693

이 파라미터 조합을 최종 모델에 적용하여 사용하세요.
